# explore


In [1]:
#| default_exp explore

In [3]:
#| export
import numpy as np

In [23]:
#| export 
class Value:
    
    def __init__(self, data: float, _prev:set=(), _op: str='') -> None:
        self.data = data
        self._prev = _prev
        self._op = _op
        
    def __repr__(self) -> str:
        return f"Value(data={self.data})"
    
    def __add__(self, other: Value) -> Value:
        return Value(self.data + other.data, _prev=(self,other),_op='+')
    
    def __mul__(self, other: Value)-> Value:
        return Value(self.data * other.data, _prev=(self,other),_op='*')

In [29]:

a = Value(2.)
b = Value(3.)
c = Value(10.)
d=c+a*b

In [36]:
d._op

'+'

In [25]:
l = [1,2,4,1]
l, set(l)
np.unique(np.array([1,2,4,1]))


array([1, 2, 4])